# PyTorch自动微分
autograd包是pytorch中所有神经网络的核心，它为tensors上的所有操作提供自动微分。它是一个由运行定义的框架，这意味着以代码运行方式定义你的后向传播，每次迭代都可以不用(动态的，和tensorflow的静态不同)

## Tensor
torch.Tensor是包的核心类。如果将其属性.requires_grad设置为True, 则会开始跟踪对tensor的所有操作。完成计算后，可以使用.backward()来自动计算所有梯度。该张量的所有梯度将会积累到.grad属性中。  

如果需要停止对tensor历史记录的追踪，则可以调用.detach()，它将其与计算历史记录分离，并防止将来的计算被追踪。  
同样如果想要停止追踪历史记录和使用了内存，还可以将代码块使用 with torch.no_grad(): 包装起来。这种操作在对模型进行评估时，十分有用，因为在训练阶段，我们需要将tensor.requires_grad = True，这样方便对训练参数进行调参，而在评估阶段我们不需要进行调参，因此则不需要梯度。

In [2]:
import torch
with torch.no_grad():
    pass

还有一个类对autograd实现非常重要，那就是Function。Tensor和Function相互连接并构建一个非循环图，他保存整个完整的计算过程的历史信息。每个张量都有一个.grad_fn属性保存着创建了张量的Function的引用，ps：如果是用户自己创建的张量，则grad_fn是None。  
如果你想要计算导数，你可调用Tensor.backward(). 如果Tensor是标量，则不需要指定任何backward(), 如果它有更多的元素，则需要制定一个gradient参数来指定张量的形状。

In [3]:
x = torch.ones(2,2, requires_grad=True) # 设置requires_grad = True 来追踪对tensor x的操作
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


针对上述张量进行操作

In [6]:
y = x + 2
print(y)
print(y.grad_fn) # 显示y的grad_fn

tensor([[3., 3.],
        [3., 3.]], grad_fn=<AddBackward0>)


针对y进行更多的操作：

In [7]:
z = y * y * 3
out = z.mean()
print(z,out)

tensor([[27., 27.],
        [27., 27.]], grad_fn=<MulBackward0>) tensor(27., grad_fn=<MeanBackward0>)


.requires_grad_()会改变张量的requires_grad标记。输入的标记默认为False。

In [15]:
a = torch.randn(2,2)
a = ((a * 3) / (a - 1))
print(a.requires_grad)
a.requires_grad_(True)
print(a.requires_grad)
b = (a * a).sum()
print(b.requires_grad)
print(b.grad_fn)
c = torch.rand(2,3)
print(c.requires_grad)

False
True
True
False


从上面例子可以看出，在没有对requires_grad进行赋值的之前默认是False，在使用requires_grad_()进行赋值之后，变为了True，同样如果将a的计算结果赋给b之后，可以看到b的requires_grad也变成了True，而c和a、b没有任何关系，因此c的requires_grad还是False。

### 梯度
现在实现对out的向后传播，并打印出来x的梯度。

In [16]:
out.backward()
print(x.grad)

tensor([[4.5000, 4.5000],
        [4.5000, 4.5000]])


这里设计的原理就是模式识别中讲到的反向传播的递推梯度计算公式了，如果不知道的可以自行去查阅了。  

下面看一个雅克比行列式向量积的例子：

In [25]:
x = torch.randn(3,requires_grad=True)
y = x * 2
print(y)
print(y.data) # 读取tensor的数值用的
while y.data.norm() < 1000:
    y = y * 2
print(y)

tensor([-0.9500,  0.0213,  2.2834], grad_fn=<MulBackward0>)
tensor([-0.9500,  0.0213,  2.2834])
tensor([-486.3931,   10.9079, 1169.0762], grad_fn=<MulBackward0>)


在这种情况下，y不再是一个标量，因为对y的计算操作会导致y进行变换。torch.autograd不能直接计算整个雅格比，但是如果我们只想要雅格比向量积，只需要简单的传递向量给backward作为参数即可。

In [26]:
v = torch.tensor([0.1,1.0,0.0001],dtype = torch.float)
y.backward(v)
print(x.grad)

tensor([1.0240e+02, 1.0240e+03, 1.0240e-01])


你也可将代码放在with torch.no_grad()中来停止对历史的跟踪中的.requires_grad=True的张量自动求导。

In [27]:
print(x.requires_grad)
print((x ** 2).requires_grad)

with torch.no_grad():
    print((x ** 2).requires_grad)

True
True
False


### tensor.norm的分析使用
这里主要介绍一些对tensor.norm()的分析和使用。官方文档中给出的使用格式如下:
```
torch.norm(input, p='fro', dim=None, keepdim=False, out=None, dtype=None)
```
先介绍一些函数的主要功能：tensor.norm()主要用于计算向量范数或者矩阵范数的，其中p给定的就是计算几范数。  
p可以给的数值：int, float, inf, -inf, 'fro', 'nuc', optional  
下面给出一个计算向量二范数的例子：

In [36]:
x = torch.tensor([3,4],dtype=torch.float)
x.requires_grad = True
x = x.data.norm(p=2) # 等价于 torch.norm(x,p=2)
print(x)

tensor(5.)


上述代码进行了向量的二范数计算，其中p设置为2。需要注意的一点是:**只有float类型的tensor才可以进行requires_grad设置进而来计算梯度**，对int类型是不支持的.  
从上述代码可以看出x.data.norm(p=2) 是等价于 torch.norm(x,p=2)